# RAG vec DB

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama

c:\Users\prana\Desktop\VS_CODE\LangChainModels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# =========================
# 2. Load Documents
# =========================
loader = PyPDFLoader(r"C:\Users\prana\Desktop\VS_CODE\LangChainModels\PDFs\Financial_Market_Securities_BSE.one.pdf")  # change to your file
documents = loader.load()


# =========================
# 3. Recursive Chunking
# =========================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)


# =========================
# 4. Embeddings (MiniLM-L6-v2)
# =========================
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


# =========================
# 5. Create FAISS Vector DB
# =========================
vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

C:\Users\prana\AppData\Local\Temp\ipykernel_12960\319920196.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [6]:
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# LLM
llm = ChatOllama(
    model="deepseek-r1:8b",
    temperature=0
)

prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)
question = "what is a preferencec Stock"
retrieved_docs    = retriever.invoke(question)

In [8]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"Preference shares are a hybrid instrument, blending features of both equity and debt. As\nthe name suggests, they offer preferential rights over equity shares. Key features include:\nDividend Priority: Preference shareholders receive dividends before equity\nshareholders. These dividends are typically at a fixed rate, providing a more\npredictable income stream compared to the variable dividends of equity shares (Angel\nOne).\nLiquidation Priority: In case of company liquidation, preference shareholders have a\nprior claim on the company's assets over equity shareholders, but after bondholders\nand other creditors.\nNo Voting Rights: Generally, preference shareholders do not have voting rights in the\ncompany, which is a key distinction from equity shareholders. However, they may gain\nvoting rights if their dividends are in arrears for a specified period (Bajaj Finserv).\nThere are various types of preference shares, such as cumulative (unpaid dividends\n\nsegments, each serving diff

In [9]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [11]:
answer = llm.invoke(final_prompt)
print(answer.content)

Based on the transcript context:

Preference shares are a hybrid instrument, blending features of both equity and debt. Key features include:

*   **Dividend Priority:** Receive dividends before equity shareholders, typically at a fixed rate.
*   **Liquidation Priority:** Have a prior claim on company assets during liquidation, after creditors but before equity shareholders.
*   **No Voting Rights:** Generally lack voting rights, although they may gain them if dividends are overdue.
*   They are considered a hybrid security.


In [12]:
answer.usage_metadata

{'input_tokens': 744, 'output_tokens': 578, 'total_tokens': 1322}